In [1]:
using
    QuantumStates,
    OpticalBlochEquations,
    DifferentialEquations,
    UnitsToValue,
    LinearAlgebra,
    Printf,
    Plots,
    Random,
    StatsBase
;

using Distributions

import MutableNamedTuples: MutableNamedTuple
import StructArrays: StructArray, StructVector
import StaticArrays: @SVector, SVector
import LinearAlgebra: norm, ⋅, adjoint!, diag
import LoopVectorization: @turbo
using BenchmarkTools
using Parameters
using LsqFit

import ProgressMeter: Progress, next!

const λ = 626e-9
const Γ = 2π* 6.4e6
const m = @with_unit 57 "u"
const k = 2π / λ
_μB = (μ_B / h) * 1e-4;

include("CaOH_scan_helper_v2.jl")
working_dir = "C:\\Google Drive\\github\\OpticalBlochEquations\\examples\\ipynb_sources\\toy_models\\CaOH_scan\\scan_3frequencies"
;

In [2]:
"""
    Function to optimize (density).
"""
function f(x)
    s1 = x[1]
    s3 = x[2]
    s4 = x[3]
    Δ1 = x[4]*Γ
    Δ3 = x[5]*Γ
    Δ4 = x[6]*Γ
    
    temp = @with_unit 0.5 "mK"
    diameter = @with_unit 0.6 "mm"
    displacement = [0.0, 0.0, 0.0]
    kick = [0,0, 0.0, 0.0]
    
    params = MutableNamedTuple(
        t_end = 30e-3, n_values = 100,
        pol1_x=σ⁻, pol2_x=σ⁺, pol3_x=σ⁻, pol4_x=σ⁺, 
        s1=s1, s2=0.0, s3=s3, s4=s4,
        Δ1=Δ1, Δ2=0.0, Δ3=Δ3, Δ4=Δ4,
        B_gradient=100,
        temp=temp, 
        diameter=diameter,
        displacement=displacement, 
        kick=kick,
        ramp_time=5e-3)
    
    results = simulate_particles_v2(package, params)
    
    return results
end
;

In [3]:
using BayesianOptimization, GaussianProcesses

model = ElasticGPE(6,                            # 6 input dimensions
                   mean = MeanConst(1.),         
                   kernel = SEArd([0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 5.),
                   logNoise = 0.,
                   capacity = 3000)              # the initial capacity of the GP is 3000 samples
set_priors!(model.mean, [Normal(1, 1)])

# Optimize the hyperparameters of the GP using maximum a posteriori (MAP) estimates every x steps
modeloptimizer = MAPGPOptimizer(
    every = 1, 
    noisebounds = [-3, 3], # bounds of the logNoise
    kernbounds = [[-3, -3, -3, -3, -3, -3, -5], [3, 3, 3, 3, 3, 3, 5]],  # bounds of the parameters GaussianProcesses.get_param_names(model.kernel)
    maxeval = 1000)

opt = BOpt(f,
           model,
           UpperConfidenceBound(),                   # type of acquisition
           modeloptimizer,                        
           [0.0, 0.0, 0.0, +0.0, -2.0, +0.0],        # lowerbounds
           [5.0, 0.3, 0.3, +4.0, +0.0, +2.0],        # upperbounds         
           repetitions = 1,                          # evaluate the function for each input x times
           maxiterations = 2000,                       # evaluate at x input positions
           sense = Max,                              # maximize the function
           acquisitionoptions = (method = :LD_LBFGS, # run optimization of acquisition function with NLopts :LD_LBFGS method
                                 restarts = 5,       # run the NLopt method from x random initial conditions each time
                                 maxtime = 1.0,      # run the NLopt method for at most 1.0 second each time
                                 maxeval = 1000),    # run the NLopt methods for at most 1000 iterations (for other options see https://github.com/JuliaOpt/NLopt.jl)
           verbosity = BayesianOptimization.Progress)
;

┌ Warning: use values(kwargs) and keys(kwargs) instead of kwargs.data and kwargs.itr
│   caller = #MAPGPOptimizer#19 at gp.jl:37 [inlined]
└ @ Core C:\Users\Christian\.julia\packages\BayesianOptimization\nq1HY\src\models\gp.jl:37


In [10]:
package = get_CaOH_package()
n_states = length(package.states)
n_excited = package.n_excited
;

In [7]:
using Serialization
opt_saved = deserialize("bayesian_optimization_save_2nd.jl")
;

In [8]:
opt.model = opt_saved.model
opt.lowerbounds = opt_saved.lowerbounds
opt.upperbounds = opt_saved.upperbounds
opt.observed_optimizer = opt_saved.observed_optimizer
opt.model_optimizer = opt_saved.model_optimizer
opt.observed_optimum = opt_saved.observed_optimum
opt.model_optimum = opt_saved.model_optimum
;

In [11]:
maxiterations!(opt, 1)
boptimize!(opt)

Progress: 100%|█████████████████████████████████████████| Time: 0:09:24


LoadError: MethodError: no method matching *(::Int64, ::MutableNamedTuple{(:x_trajectories, :y_trajectories, :z_trajectories, :x_velocities, :y_velocities, :z_velocities, :times, :A_populations, :n_values, :photons_scattered), Tuple{Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Vector{Vector{Float64}}}, Base.RefValue{Int64}, Base.RefValue{Vector{Float64}}}})

[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:578[24m[39m
[0m  *(::T, [91m::T[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}
[0m[90m   @[39m [90mBase[39m [90m[4mint.jl:88[24m[39m
[0m  *(::Union{Bool, Float16, Float32, Float64, Int